In [21]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from itertools import product

df = pd.read_csv("data/npf_train.csv")


df["event"], _ = pd.factorize(df["class4"])
df = df.drop(["class4", "partlybad"], axis = 1)
y = df["event"].values

#Remove columns that we don't want to use.
columns = df.columns.values.tolist()
remove = ["id", "date", "event"]
columns = [column for column in columns if column not in remove]

df = df[columns].apply(pd.to_numeric, errors='coerce')

df_pairs = pd.DataFrame(columns = np.arange(10000), index= np.arange(df.shape[0]))

def pairs(row):
    return [(x[0] * x[1]) for x in product(row, repeat= 2)]

for index, row in df.iterrows():
    df_pairs.iloc[index, 0:10000] = pairs(row)



row = pairs(df.iloc[0].values)
print(len(df.iloc[0].values))
print(len(row))
df_pairs

model = Pipeline([
    ('minmax', MinMaxScaler()),
    ('model', MultinomialNB()),
])

X = df_pairs.values

X, X_test, y, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
X_train, X_validate, y_train, y_validate = train_test_split(X, y, train_size= 0.75, random_state= 42)

model.fit(X_train, y_train)

y_pred = model.predict(X_validate)

print("accuracy: ", accuracy_score(y_validate, y_pred))
print("cross validate score: ", cross_validate(model, X_train, y_train, cv = 3))

100
10000
accuracy:  0.5698924731182796
cross validate score:  {'fit_time': array([0.28724742, 0.29282403, 0.30543733]), 'score_time': array([0.09474921, 0.08274698, 0.08378005]), 'test_score': array([0.65591398, 0.55913978, 0.52173913])}
